# PROYECTO INDIVIDUAL 1 MLOps: endpoints

## 1) Importación de librerías

In [83]:
%matplotlib inline
import numpy as np
import pandas as pd
from scipy.stats import skew
from math import sqrt
from numpy import mean, var
import ast
import re

#import matplotlib.pyplot as plt 
#import scipy
#from scipy import stats
#import copy 
#from sklearn import preprocessing
#import seaborn as sns
#import json

## 2) Lectura de archivo csv como dataframe

In [84]:
# Leemos el dataset y le asignamos el nombre 'df', mostrmoas las 5 primeras filas para ver el formato de cada columna.
df=pd.read_csv('movies_dataset.csv') 
df.head() 

C:\Users\Usuario\AppData\Local\Temp\ipykernel_832\2226068471.py:2: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df=pd.read_csv('movies_dataset.csv')


,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


In [85]:
# Analizamos el tamaño del dataset
df.shape

(45466, 24)

## 3) Transformaciones

### T1: Expansión de columnas anidadas

In [86]:
# Definimos una expresión regular para extraer el nombre de la colección
pattern = r"'name': '([^']*)'"

# Extraemos el nombre de la colección utilizando la expresión regular
df['collection_name'] = df['belongs_to_collection'].str.extract(pattern, flags=re.IGNORECASE)

# Imprimimos el DataFrame con la columna normalizada
df.head()

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,collection_name
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0,Toy Story Collection
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0,NaN
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0,Grumpy Old Men Collection
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0,NaN
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0,Father of the Bride Collection


In [87]:
# Vemos los distintos valores de coleciones
df.collection_name.unique()

array(['Toy Story Collection', nan, 'Grumpy Old Men Collection', ...,
       'Ducobu Collection', 'Mister Blot Collection',
       'Red Lotus Collection'], dtype=object)

In [88]:
# No muestra todos los valores, por lo que los contamos:
df.collection_name.nunique()

1644

In [89]:
# Crear una nueva columna para los registros duplicados
df['collection_individual'] = df['collection_name'].str.split(', ')

# Duplicar los registros
df = df.explode('collection_individual').reset_index(drop=True)

# Eliminar la columna original "collection_name"
df.drop('collection_name', axis=1, inplace=True)

In [90]:
df.collection_individual.unique()

array(['Toy Story Collection', nan, 'Grumpy Old Men Collection', ...,
       'Ducobu Collection', 'Mister Blot Collection',
       'Red Lotus Collection'], dtype=object)

In [91]:
df.collection_individual.nunique()

1667

In [92]:
# Aplicamos el mismo proceso para el resto de las columnas con valores tipo diccionario o listas:
# Convertimos la cadena JSON en una lista de diccionarios
df['genres'] = df['genres'].apply(ast.literal_eval)

# Función personalizada para extraer los nombres de los géneros
def extract_genre_names(genres_list):
    genre_names = [genre['name'] for genre in genres_list]
    return ', '.join(genre_names)

# Aplicamos la función personalizada a cada elemento de la columna "genres"
df['genre'] = df['genres'].apply(extract_genre_names)

# Imprimimos el DataFrame con la columna de géneros ('genre') normalizados
df.head()

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,collection_individual,genre
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0,Toy Story Collection,"Animation, Comedy, Family"
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0,NaN,"Adventure, Fantasy, Family"
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0,Grumpy Old Men Collection,"Romance, Comedy"
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0,NaN,"Comedy, Drama, Romance"
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0,Father of the Bride Collection,Comedy


In [93]:
# Vemos todos los valores posibles de la columna 'genre'
df.genre.unique()


array(['Animation, Comedy, Family', 'Adventure, Fantasy, Family',
       'Romance, Comedy', ..., 'Crime, Comedy, Action, Family',
       'Action, Mystery, Thriller, Horror',
       'Family, Animation, Romance, Comedy'], dtype=object)

In [94]:
# No muestra todos, por lo que contaremos los valores sin repetir
df.genre.nunique()
# Vemos que al agrupar varios generos en una misma lista hay muchas combinaciones posibles, por lo que es correcto separar cada genero como un único elemento de una lista.

4069

In [95]:
# Crear una nueva columna para los registros duplicados
df['genre_individual'] = df['genre'].str.split(', ')

# Duplicar los registros
df = df.explode('genre_individual').reset_index(drop=True)

# Eliminar la columna original "genre"
df.drop('genre', axis=1, inplace=True)

In [96]:
df.genre_individual.unique()
# Ahora podemos ver todos los tipos posibles de géneros en el dataset

array(['Animation', 'Comedy', 'Family', 'Adventure', 'Fantasy', 'Romance',
       'Drama', 'Action', 'Crime', 'Thriller', 'Horror', 'History',
       'Science Fiction', 'Mystery', 'War', 'Foreign', '', 'Music',
       'Documentary', 'Western', 'TV Movie', 'Carousel Productions',
       'Vision View Entertainment', 'Telescene Film Group Productions',
       'Aniplex', 'GoHands', 'BROSTA TV',
       'Mardock Scramble Production Committee', 'Sentai Filmworks',
       'Odyssey Media', 'Pulser Productions', 'Rogue State', 'The Cartel'],
      dtype=object)

In [97]:
# Contamos los tipos de genero
df.genre_individual.nunique()

33

In [98]:
# Función personalizada para normalizar la columna "production_companies"
def normalize_production_companies(value):
    try:
        pattern = r"'name':\s+'(.*?)'"
        matches = re.findall(pattern, value)
        return ', '.join(matches)
    except (TypeError, AttributeError):
        return ''

# Aplicamos la función personalizada a cada elemento de la columna "production_companies"
df['company'] = df['production_companies'].apply(normalize_production_companies)

# Crear una nueva columna para los registros duplicados
df['company_individual'] = df['company'].str.split(', ')

# Duplicar los registros
df = df.explode('company_individual').reset_index(drop=True)

# Eliminar la columna original "company"
df.drop('company', axis=1, inplace=True)


In [99]:
# Función personalizada para normalizar la columna "production_countries"
def normalize_production_countries(value):
    try:
        pattern = r"'name':\s+'(.*?)'"
        matches = re.findall(pattern, value)
        return ', '.join(matches)
    except (TypeError, AttributeError):
        return ''

# Aplicamos la función personalizada a cada elemento de la columna "production_countries"
df['country'] = df['production_countries'].apply(normalize_production_countries)

# Crear una nueva columna para los registros duplicados
df['country_individual'] = df['country'].str.split(', ')

# Duplicar los registros
df = df.explode('country_individual').reset_index(drop=True)

# Eliminar la columna original "country"
df.drop('country', axis=1, inplace=True)

In [100]:
# Función personalizada para normalizar la columna "spoken_languages"
def normalize_spoken_languages(value):
    try:
        pattern = r"'name':\s+'(.*?)'"
        matches = re.findall(pattern, value)
        return ', '.join(matches)
    except (TypeError, AttributeError):
        return ''

# Aplicamos la función personalizada a cada elemento de la columna "spoken_languages"
df['spoken_language'] = df['spoken_languages'].apply(normalize_spoken_languages)

# Crear una nueva columna para los registros duplicados
df['language_individual'] = df['spoken_language'].str.split(', ')

# Duplicar los registros
df = df.explode('language_individual').reset_index(drop=True)

# Eliminar la columna original "spoken_language"
df.drop('spoken_language', axis=1, inplace=True)

In [101]:
# Eliminamos de df las columnas de donde creamos las columnas nuevas y vemos los primeros valores del df 
df.drop(columns=['belongs_to_collection', 'genres', 'production_companies', 'production_countries', 'spoken_languages'], inplace = True)

# Renombramos las columnas creadas
df.rename(columns={ 'collection_individual': 'collection', 
                    'genre_individual': 'genre',
                    'company_individual': 'company',
                    'country_individual': 'country',
                    'language_individual': 'language'
}, inplace=True)
df.columns

Index(['adult', 'budget', 'homepage', 'id', 'imdb_id', 'original_language',
       'original_title', 'overview', 'popularity', 'poster_path',
       'release_date', 'revenue', 'runtime', 'status', 'tagline', 'title',
       'video', 'vote_average', 'vote_count', 'collection', 'genre', 'company',
       'country', 'language'],
      dtype='object')

In [102]:
df.shape

(419666, 24)

### T2: Tratamiento de valores nulos

In [103]:
# Los valores nulos de los campos revenue, budget son rellenados por el número 0
df['revenue'].fillna(0, inplace=True)
df['budget'].fillna(0, inplace=True)

print('Nulos en revenue',df.revenue.isna().sum())
print('Nulos en budget',df.revenue.isna().sum())

Nulos en revenue 0
Nulos en budget 0


### T3: Modificaciones en datos de tipo fecha

In [104]:
# De haber fechas, deberán tener el formato AAAA-mm-dd, además deberán crear la columna release_year donde extraerán el año de la fecha de 
# estreno.
# Le damos el formato fecha a todos los valores de la columna y los que no, le asignamos la etiqueta de nulo
df['release_date'] = pd.to_datetime(df['release_date'], errors='coerce')

# Se asegura que todos los registros tengan el mismo formato de fecha AAAA-MM-DD
df['release_date'] = pd.to_datetime(df['release_date']).dt.strftime('%Y-%m-%d')

# Se identifican la cantidad de datos nulos de la columna 'release_date' 
df['release_date'].isnull().sum()
# Eliminamos los registros con nulos 
df = df.dropna(subset = ['release_date'])

# Controlamos para verificar que no existan nulos
print("Nulos en release_date:", df['release_date'].isnull().sum())
# Creamos una nueva columna 'release_year' que va a contener unicamnete el dato de año de la columna 'release_date' 
df['release_date'] = pd.to_datetime(df['release_date'])
df['release_year']=df['release_date'].dt.year
df.head(3)

Nulos en release_date: 0


,adult,budget,homepage,id,imdb_id,original_language,original_title,overview,popularity,poster_path,...,title,video,vote_average,vote_count,collection,genre,company,country,language,release_year
0,False,30000000,http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.946943,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,...,Toy Story,False,7.7,5415.0,Toy Story Collection,Animation,Pixar Animation Studios,United States of America,English,1995
1,False,30000000,http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.946943,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,...,Toy Story,False,7.7,5415.0,Toy Story Collection,Comedy,Pixar Animation Studios,United States of America,English,1995
2,False,30000000,http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.946943,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,...,Toy Story,False,7.7,5415.0,Toy Story Collection,Family,Pixar Animation Studios,United States of America,English,1995


### T4: Nueva columna Return

In [105]:
# Creamos la columna con el retorno de inversión, llamada return con los campos revenue y budget, dividiendo estas dos últimas revenue / budget, 
# Cuando no hay datos disponibles para calcularlo, deberá tomar el valor 0

# Convertimos 'revenue' y 'budget' al tipo numérico (float)
df['revenue'] = pd.to_numeric(df['revenue'], errors='coerce')
df['budget'] = pd.to_numeric(df['budget'], errors='coerce')

# Calculamos el retorno de inversión
df['return'] = df['revenue'] / df['budget']
df['return'].fillna(0, inplace=True)
df['return']

0         12.451801
1         12.451801
2         12.451801
3          4.043035
4          4.043035
            ...    
419661     0.000000
419662     0.000000
419663     0.000000
419664     0.000000
419665     0.000000
Name: return, Length: 419475, dtype: float64

### T5: Eliminación de columnas innecesarias

In [106]:
# Eliminamos las columnas que no serán utilizadas, video,imdb_id,adult,original_title,vote_count,poster_path y homepage
df.drop(columns=['video', 'imdb_id', 'adult', 'original_title', 'vote_count', 'poster_path', 'homepage'], inplace = True)

In [107]:
df.columns

Index(['budget', 'id', 'original_language', 'overview', 'popularity',
       'release_date', 'revenue', 'runtime', 'status', 'tagline', 'title',
       'vote_average', 'collection', 'genre', 'company', 'country', 'language',
       'release_year', 'return'],
      dtype='object')

## 4) Endpoints

#### E1: peliculas_mes

In [108]:
# 1) def peliculas_mes(mes): Se ingresa el mes(en palabra) y la funcion retorna la cantidad de peliculas que se estrenaron ese mes historicamente - return {'mes':mes, 'cantidad':respuesta}
def peliculas_mes(mes):
    '''Se ingresa el mes(en palabra) y la funcion retorna la cantidad de peliculas que se estrenaron ese mes historicamente.'''
    mes = mes.lower()
    meses = {
    'enero': 1,
    'febrero': 2,
    'marzo': 3,
    'abril': 4,
    'mayo': 5,
    'junio': 6,
    'julio': 7,
    'agosto': 8,
    'septiembre': 9,
    'octubre': 10,
    'noviembre': 11,
    'diciembre': 12}

    mes_numero = meses[mes]

    # Convertir la columna "fecha" a un objeto de tipo fecha
    df['release_date'] = pd.to_datetime(df['release_date'])

    try:
        month_filtered = df[df['release_date'].dt.month == mes_numero]
    except (ValueError, KeyError, TypeError):
        return None
    
    month_unique = month_filtered.drop_duplicates(subset='id')
    respuesta = month_unique.shape[0]
    return {'mes':mes, 'cantidad':respuesta}

In [109]:
peliculas_mes('junio')

{'mes': 'junio', 'cantidad': 3151}

#### E2: peliculas_dia

In [110]:
# 2) def peliculas_dia(dia): Se ingresa el dia y la funcion retorna la cantidad de peliculas que se estrebaron ese dia historicamente return {'dia':dia, 'cantidad':respuesta}
def peliculas_dia(dia):
    '''Se ingresa el dia de la semana y la funcion retorna la cantidad de peliculas que se estrebaron ese dia historicamente.'''
    days = {
    'lunes': 'Monday',
    'martes': 'Tuesday',
    'miercoles': 'Wednesday',
    'jueves': 'Thursday',
    'viernes': 'Friday',
    'sabado': 'Saturday',
    'domingo': 'Sunday'}

    day = days[dia.lower()]
    
    lista_peliculas_day = df[df['release_date'].dt.day_name() == day].drop_duplicates(subset='id')
    cantidad = lista_peliculas_day.shape[0]

    return {'dia': dia, 'cantidad': cantidad}

In [111]:
peliculas_dia('miercoles')

{'dia': 'miercoles', 'cantidad': 7027}

### E3: franquicia

In [112]:
# 3) def franquicia(franquicia): Se ingresa la franquicia, retornando la cantidad de peliculas, ganancia total y promedio return 
# {'franquicia':franquicia, 'cantidad':respuesta, 'ganancia_total':respuesta, 'ganancia_promedio':respuesta}
def franquicia(franquicia):
    '''Se ingresa la franquicia, retornando la cantidad de peliculas, ganancia total y promedio de ganancias.'''
    lista_pelis_franquicia = df[(df['collection'] == franquicia)].drop_duplicates(subset='id')

    cantidad_pelis_franq = (lista_pelis_franquicia).shape[0]
    revenue_franq = lista_pelis_franquicia['revenue'].sum()
    promedio_franq = revenue_franq/cantidad_pelis_franq

    return {'franquicia':franquicia, 'cantidad':cantidad_pelis_franq, 'ganancia_total':revenue_franq, 'ganancia_promedio':promedio_franq}

In [113]:
franquicia('Star Wars Collection')

{'franquicia': 'Star Wars Collection',
 'cantidad': 8,
 'ganancia_total': 7434494790.0,
 'ganancia_promedio': 929311848.75}

### E4: peliculas_pais

In [114]:
# 4) def peliculas_pais(pais): Ingresas el pais, retornando la cantidad de peliculas producidas en el mismo return {'pais':pais, 'cantidad':respuesta}
def peliculas_pais(pais):
    '''Ingresas el pais, retornando la cantidad de peliculas producidas en el mismo.'''
    movies_filtered = df[(df['country'] == pais)]
    movies_unique = movies_filtered.drop_duplicates(subset='id')
    movie_count = movies_unique.shape[0]
    return {'pais':pais, 'cantidad':movie_count}

In [115]:
peliculas_pais('Argentina')

{'pais': 'Argentina', 'cantidad': 254}

### E5: productoras

In [116]:
# 5) def productoras(productora): Ingresas la productora, retornando la ganancia total y la cantidad de peliculas que produjeron return 
# {'productora':productora, 'ganancia_total':respuesta, 'cantidad':respuesta}
def productoras(productora):
    '''Ingresas la productora, retornando la ganancia total y la cantidad de peliculas que produjeron.'''
    lista_pelis_productoras = df[(df['company'] == productora)].drop_duplicates(subset='id')

    cantidad_pelis_prod = (lista_pelis_productoras).shape[0]
    revenue_prod = lista_pelis_productoras['revenue'].sum()

    return {'productora':productora, 'ganancia_total':revenue_prod, 'cantidad':cantidad_pelis_prod}

In [117]:
productoras('Universal Studios')

{'productora': 'Universal Studios',
 'ganancia_total': 3265387180.0,
 'cantidad': 34}

### E6: retorno

In [118]:
# 6) def retorno(pelicula): Ingresas la pelicula, retornando la inversion, la ganancia, el retorno y el año en el que se lanzo return 
# {'pelicula':pelicula, 'inversion':respuesta, 'ganacia':respuesta,'retorno':respuesta, 'anio':respuesta}
def retorno(pelicula):
    '''Ingresas la pelicula, retornando la inversion, la ganancia, el retorno y el año en el que se lanzo.'''
    info_pelicula = df[(df['title'] == pelicula)].drop_duplicates(subset='title')
    pelicula_nombre = info_pelicula['title'].iloc[0]
    inversion_pelicula = str(info_pelicula['budget'].iloc[0])
    ganancia_pelicula = str(info_pelicula['revenue'].iloc[0])
    retorno_pelicula = str(info_pelicula['return'].iloc[0])
    year_pelicula = str(info_pelicula['release_year'].iloc[0])

    return {'pelicula':pelicula_nombre, 'inversion':inversion_pelicula, 'ganacia':ganancia_pelicula,'retorno':retorno_pelicula, 'anio':year_pelicula}

In [119]:
retorno('Titanic')

{'pelicula': 'Titanic',
 'inversion': '200000000',
 'ganacia': '1845034188.0',
 'retorno': '9.22517094',
 'anio': '1997'}

## 5) Exporto csv

In [121]:
#Exporto el df a csv para luego utilizarlo en la API.
df.to_csv('movies_endpoints.csv', index=False)